# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = False
save_masks = False
print_loss = True
output_tensors = False
EPOCHS = 5
iteration = "-14"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 50
#val_batch_size = 100
num_batch = 2000 #25000 #11112
val_batch = 2
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    # time
    since = time.time()
    best_model = None
    best_loss = math.inf
    
    outputs_dir = "data/outputs/results/" + str(datetime.now().date())
    os.makedirs(outputs_dir, exist_ok=True)
    
    # logs
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    batch_log_name = str(datetime.now().date()) + iteration
    epoch_log_name = "epoch - "+ str(datetime.now().date()) + iteration
    
    # model and sigmoid function
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
            
        epoch_loss = 0
        
        #Training    
        model.train()
        phase = 'train'
        for batch in range(num_batch):
            if print_loss:
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            running_loss = 0.0
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)

            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                batch_log.write(log_dir, batch_log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= num_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, val_batch), flush=True)
            
            #BATCH TUPLE
#           inputs, labels, paths = next(iter(train_dataloaders))
            inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)

            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                batch_log.write(log_dir, batch_log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
            
            if save_masks and phase == 'val':
                outputs = sigmoid_function(outputs)
                j = 0
                for locations in paths:
                    img = Image.open(locations)
#                     num = locations.replace("./data/train/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
                    for threshold in [0.05, .25, .5, .75]:
                        img = data_factory.showMaskOnImage(img,outputs[j], threshold)
                        img.save(outputs_dir  + "/train-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
                    j += 1
                
        val_loss /= val_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, epoch_log_name)    

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model, LOAD_LOCATION)
            torch.save(model, SAVE_LOCATION)
            torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
        
    #save current model
    torch.save(model,SAVE_LOCATION)
    batch_log.write(log_dir, batch_log_name)
    epoch_log.write(log_dir, epoch_log_name)
    
    
    time_elapsed = time.time() - since
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = MyNet1(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    print('Single convolution model, no weights', flush=True)
#     model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
#     print('initializing model with random weights', flush=True)
#     torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)
#     torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.15, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  

loading weights from ./data/models/model_best


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/3
Batch 1/2000
loss: 0.6914117336273193
---------------
Batch 2/2000
loss: 0.6906211972236633
---------------
Batch 3/2000
loss: 0.6911913752555847
---------------
Batch 4/2000
loss: 0.6907337307929993
---------------
Batch 5/2000
loss: 0.6918307542800903
---------------
Batch 6/2000
loss: 0.6884799003601074
---------------
Batch 7/2000
loss: 0.6904330849647522
---------------
Batch 8/2000
loss: 0.6868559122085571
---------------
Batch 9/2000
loss: 0.6908628940582275
---------------
Batch 10/2000
loss: 0.6928357481956482
---------------
Batch 11/2000
loss: 0.6907097697257996
---------------
Batch 12/2000
loss: 0.6904807686805725
---------------
Batch 13/2000
loss: 0.692624568939209
---------------
Batch 14/2000
loss: 0.6919996738433838
---------------
Batch 15/2000
loss: 0.6902844905853271
---------------
Batch 16/2000
loss: 0.6927401423454285
---------------
Batch 17/2000
loss: 0.690473735332489
---------------
Batch 18/2000
loss: 0.6923555135726929
---------------
Batch 19/20

---------------
Batch 150/2000
loss: 0.6939794421195984
---------------
Batch 151/2000
loss: 0.6932277679443359
---------------
Batch 152/2000
loss: 0.6912303566932678
---------------
Batch 153/2000
loss: 0.6910751461982727
---------------
Batch 154/2000
loss: 0.6921419501304626
---------------
Batch 155/2000
loss: 0.6900269985198975
---------------
Batch 156/2000
loss: 0.6892289519309998
---------------
Batch 157/2000
loss: 0.6910043358802795
---------------
Batch 158/2000
loss: 0.6900865435600281
---------------
Batch 159/2000
loss: 0.6898244619369507
---------------
Batch 160/2000
loss: 0.6909745335578918
---------------
Batch 161/2000
loss: 0.6894482970237732
---------------
Batch 162/2000
loss: 0.6918206214904785
---------------
Batch 163/2000
loss: 0.6896383762359619
---------------
Batch 164/2000
loss: 0.6900913119316101
---------------
Batch 165/2000
loss: 0.688853919506073
---------------
Batch 166/2000
loss: 0.6898173093795776
---------------
Batch 167/2000
loss: 0.6888723969

---------------
Batch 297/2000
loss: 0.6896534562110901
---------------
Batch 298/2000
loss: 0.690717339515686
---------------
Batch 299/2000
loss: 0.6904222965240479
---------------
Batch 300/2000
loss: 0.6900630593299866
---------------
Batch 301/2000
loss: 0.6930626034736633
---------------
Batch 302/2000
loss: 0.6907665133476257
---------------
Batch 303/2000
loss: 0.6879022717475891
---------------
Batch 304/2000
loss: 0.6924631595611572
---------------
Batch 305/2000
loss: 0.6898462176322937
---------------
Batch 306/2000
loss: 0.6902213096618652
---------------
Batch 307/2000
loss: 0.6919735074043274
---------------
Batch 308/2000
loss: 0.6885903477668762
---------------
Batch 309/2000
loss: 0.6914931535720825
---------------
Batch 310/2000
loss: 0.6910693645477295
---------------
Batch 311/2000
loss: 0.6898084878921509
---------------
Batch 312/2000
loss: 0.6908233165740967
---------------
Batch 313/2000
loss: 0.6893358826637268
---------------
Batch 314/2000
loss: 0.6909216642

---------------
Batch 444/2000
loss: 0.690697431564331
---------------
Batch 445/2000
loss: 0.6903796792030334
---------------
Batch 446/2000
loss: 0.6905128955841064
---------------
Batch 447/2000
loss: 0.6910603046417236
---------------
Batch 448/2000
loss: 0.6914832592010498
---------------
Batch 449/2000
loss: 0.6924709677696228
---------------
Batch 450/2000
loss: 0.691307783126831
---------------
Batch 451/2000
loss: 0.6921945214271545
---------------
Batch 452/2000
loss: 0.6905904412269592
---------------
Batch 453/2000
loss: 0.6915150284767151
---------------
Batch 454/2000
loss: 0.6909800171852112
---------------
Batch 455/2000
loss: 0.6905168294906616
---------------
Batch 456/2000
loss: 0.6902744770050049
---------------
Batch 457/2000
loss: 0.6907219290733337
---------------
Batch 458/2000
loss: 0.6930116415023804
---------------
Batch 459/2000
loss: 0.6882390975952148
---------------
Batch 460/2000
loss: 0.6914206743240356
---------------
Batch 461/2000
loss: 0.69176363945

---------------
Batch 591/2000
loss: 0.689251720905304
---------------
Batch 592/2000
loss: 0.6908116340637207
---------------
Batch 593/2000
loss: 0.6917538642883301
---------------
Batch 594/2000
loss: 0.6917200684547424
---------------
Batch 595/2000
loss: 0.6905834078788757
---------------
Batch 596/2000
loss: 0.6901479959487915
---------------
Batch 597/2000
loss: 0.6924023032188416
---------------
Batch 598/2000
loss: 0.6907163858413696
---------------
Batch 599/2000
loss: 0.6908511519432068
---------------
Batch 600/2000
loss: 0.6905858516693115
---------------
Batch 601/2000
loss: 0.691647469997406
---------------
Batch 602/2000
loss: 0.6937160491943359
---------------
Batch 603/2000
loss: 0.6921595931053162
---------------
Batch 604/2000
loss: 0.6900631189346313
---------------
Batch 605/2000
loss: 0.6925088763237
---------------
Batch 606/2000
loss: 0.6891564726829529
---------------
Batch 607/2000
loss: 0.6907055974006653
---------------
Batch 608/2000
loss: 0.69149565696716

...

loss: 0.690602719783783
---------------
Batch 1394/2000
loss: 0.6897149682044983
---------------
Batch 1395/2000
loss: 0.691688597202301
---------------
Batch 1396/2000
loss: 0.6905584335327148
---------------
Batch 1397/2000
loss: 0.6908314228057861
---------------
Batch 1398/2000
loss: 0.6901456713676453
---------------
Batch 1399/2000
loss: 0.6914226412773132
---------------
Batch 1400/2000
loss: 0.6919127702713013
---------------
Batch 1401/2000
loss: 0.6911779046058655
---------------
Batch 1402/2000
loss: 0.6916447877883911
---------------
Batch 1403/2000
loss: 0.6927207708358765
---------------
Batch 1404/2000
loss: 0.6909325122833252
---------------
Batch 1405/2000
loss: 0.6916772127151489
---------------
Batch 1406/2000
loss: 0.692159116268158
---------------
Batch 1407/2000
loss: 0.6918659210205078
---------------
Batch 1408/2000
loss: 0.6909665465354919
---------------
Batch 1409/2000
loss: 0.6921143531799316
---------------
Batch 1410/2000
loss: 0.6907750368118286
---------

---------------
Batch 1538/2000
loss: 0.6919634342193604
---------------
Batch 1539/2000
loss: 0.6896320581436157
---------------
Batch 1540/2000
loss: 0.6907528638839722
---------------
Batch 1541/2000
loss: 0.690572202205658
---------------
Batch 1542/2000
loss: 0.6886191368103027
---------------
Batch 1543/2000
loss: 0.6906951665878296
---------------
Batch 1544/2000
loss: 0.6911513209342957
---------------
Batch 1545/2000
loss: 0.6900187134742737
---------------
Batch 1546/2000
loss: 0.6889066696166992
---------------
Batch 1547/2000
loss: 0.6922627687454224
---------------
Batch 1548/2000
loss: 0.6934841871261597
---------------
Batch 1549/2000
loss: 0.6921712160110474
---------------
Batch 1550/2000
loss: 0.6907366514205933
---------------
Batch 1551/2000
loss: 0.6896127462387085
---------------
Batch 1552/2000
loss: 0.6896592974662781
---------------
Batch 1553/2000
loss: 0.6922387480735779
---------------
Batch 1554/2000
loss: 0.6918039917945862
---------------
Batch 1555/2000


Batch 1682/2000
loss: 0.6917033195495605
---------------
Batch 1683/2000
loss: 0.6920148134231567
---------------
Batch 1684/2000
loss: 0.692449152469635
---------------
Batch 1685/2000
loss: 0.6915743350982666
---------------
Batch 1686/2000
loss: 0.6902199387550354
---------------
Batch 1687/2000
loss: 0.6910692453384399
---------------
Batch 1688/2000
loss: 0.6919869780540466
---------------
Batch 1689/2000
loss: 0.6908593773841858
---------------
Batch 1690/2000
loss: 0.6912384033203125
---------------
Batch 1691/2000
loss: 0.6897984743118286
---------------
Batch 1692/2000
loss: 0.6892706155776978
---------------
Batch 1693/2000
loss: 0.6899095773696899
---------------
Batch 1694/2000
loss: 0.6899610161781311
---------------
Batch 1695/2000
loss: 0.6900463104248047
---------------
Batch 1696/2000
loss: 0.6906220316886902
---------------
Batch 1697/2000
loss: 0.6905732154846191
---------------
Batch 1698/2000
loss: 0.694421112537384
---------------
Batch 1699/2000
loss: 0.694300711

loss: 0.6924257874488831
---------------
Batch 1827/2000
loss: 0.689378023147583
---------------
Batch 1828/2000
loss: 0.689887523651123
---------------
Batch 1829/2000
loss: 0.6921795010566711
---------------
Batch 1830/2000
loss: 0.6894196271896362
---------------
Batch 1831/2000
loss: 0.690619945526123
---------------
Batch 1832/2000
loss: 0.6896314024925232
---------------
Batch 1833/2000
loss: 0.6906169652938843
---------------
Batch 1834/2000
loss: 0.6920776963233948
---------------
Batch 1835/2000
loss: 0.6917157769203186
---------------
Batch 1836/2000
loss: 0.6915213465690613
---------------
Batch 1837/2000
loss: 0.6900679469108582
---------------
Batch 1838/2000
loss: 0.6924142241477966
---------------
Batch 1839/2000
loss: 0.6904482841491699
---------------
Batch 1840/2000
loss: 0.692688524723053
---------------
Batch 1841/2000
loss: 0.6892145872116089
---------------
Batch 1842/2000
loss: 0.6897204518318176
---------------
Batch 1843/2000
loss: 0.6908287405967712
----------

---------------
Batch 1971/2000
loss: 0.6905478239059448
---------------
Batch 1972/2000
loss: 0.6886749863624573
---------------
Batch 1973/2000
loss: 0.6904447674751282
---------------
Batch 1974/2000
loss: 0.6905069351196289
---------------
Batch 1975/2000
loss: 0.6895571947097778
---------------
Batch 1976/2000
loss: 0.693122923374176
---------------
Batch 1977/2000
loss: 0.6908257007598877
---------------
Batch 1978/2000
loss: 0.6908852458000183
---------------
Batch 1979/2000
loss: 0.6909676790237427
---------------
Batch 1980/2000
loss: 0.6900409460067749
---------------
Batch 1981/2000
loss: 0.6904749870300293
---------------
Batch 1982/2000
loss: 0.6910611391067505
---------------
Batch 1983/2000
loss: 0.6910253167152405
---------------
Batch 1984/2000
loss: 0.6904829740524292
---------------
Batch 1985/2000
loss: 0.6914312839508057
---------------
Batch 1986/2000
loss: 0.6899513006210327
---------------
Batch 1987/2000
loss: 0.6892725229263306
---------------
Batch 1988/2000


# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [8]:
#save state model as model
model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
model.load_state_dict(torch.load(LOAD_LOCATION+'_test-2019-04-23-3'))
torch.save(model, LOAD_LOCATION)